# add c_EC as one of the inputs for diffusivity and conductivity; use exponeitial extrapolation for Nyman2008 and make the added function a Guass rather than tanh

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import traceback

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1
    sigma_e = (
        (0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2)) * 
        (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )  )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    coff = 1
    D_c_e = (
        ( 0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2) )  * 
        (6 * exp( -1 *(c_e/1000)) 
        * exp(-5/298.15) 
        * exp(-95/298.15*(c_e/1000)) * 1e-10 ) )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = "[input]" # 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =     "[input]" # -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =   "[input]" # 5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


In [6]:
Xi =  list(np.linspace(-1.8, -1.0, 10))
print(type(Xi))

<class 'list'>


In [15]:
len(Set_succ)
len(Set_fail)

45

In [ ]:
Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 3.750125e-10

In [34]:
Sol_ddiff    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.4,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":5e-14,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

In [13]:
Sim_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
index_c = 5
Xi =  list(np.linspace(-1.8, -1.0, index_c));
t_0plus = list(np.linspace(0.01, 0.3, index_c));
D_cross = list(np.linspace(1.5e-18, 1.5e-11,index_c));
D_ec = list(np.linspace(5e-14, 5e-10, index_c));
Set_succ = []; Set_fail = []; ErrorMess = [];
t_0plus_i = 0.28;  D_ec_i = 5e-10; 
for Xi_i in Xi:
    for t_0plus_i in t_0plus:
        for D_cross_i in D_cross:
                for D_ec_i in D_ec:
                    try:
                        Sol_ddiff    = Sim_ddiff.solve(
                            inputs={
                                "EC transference number":Xi_i,
                                "Cation transference number":t_0plus_i,
                                "EC Lithium ion cross diffusivity [m2.s-1]":D_cross_i,
                                "EC diffusivity in electrolyte [m2.s-1]":D_ec_i,});
                    except:
                        str_error = traceback.format_exc()  
                        print(str_error)
                        ErrorMess.append(str_error)
                        print("Fail for: ", 
                        'Xi=',Xi_i, "t_0plus=",t_0plus_i, 
                        "D_cross=",D_cross_i, "D_ec=", D_ec_i );
                        Set_fail.append([Xi_i,t_0plus_i,D_cross_i,D_ec_i])

                    else:
                        print("Succeed for: ", 
                        'Xi=',Xi_i, "t_0plus=",t_0plus_i, 
                        "D_cross=",D_cross_i, "D_ec=", D_ec_i );
                        Set_succ.append([Xi_i,t_0plus_i,D_cross_i,D_ec_i])
            

Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 5e-14
Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 1.250375e-10


The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.


Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 2.50025e-10


The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
2022-10-06 13:00:33.848 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Maximum number of decreased steps occurred at t=3057.756394458103. Try solving the model up to this time only or reducing dt_max (currently, dt_max=7.091632984777611) and/or reducing the size of the time steps or period of the experiment.


Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 3.750125e-10


...

Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.1250000375e-11 D_ec= 5e-14
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.1250000375e-11 D_ec= 1.250375e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.1250000375e-11 D_ec= 2.50025e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.1250000375e-11 D_ec= 3.750125e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.1250000375e-11 D_ec= 5e-10


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
At t = 0.00118884 and h = 2.77904e-016, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
At t = 0.00118561 and h = 3.94023e-015, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
At t = 0.0011828 and h = 2.94795e-017, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
psetup failed: .../casadi/interfaces/sundials/idas_

Traceback (most recent call last):
  File "C:\Users\rl1120\AppData\Local\Temp\ipykernel_16444\920439172.py", line 16, in <cell line: 11>
    Sol_ddiff    = Sim_ddiff.solve(
  File "d:\gitpybamm\lrh_409\lrh_409\envsolcon\pybamm-gem\pybamm\simulation.py", line 843, in solve
    raise e
  File "d:\gitpybamm\lrh_409\lrh_409\envsolcon\pybamm-gem\pybamm\simulation.py", line 829, in solve
    step_solution = solver.step(
  File "d:\gitpybamm\lrh_409\lrh_409\envsolcon\pybamm-gem\pybamm\solvers\base_solver.py", line 1322, in step
    solution = self._integrate(model, t_eval, ext_and_inputs)
  File "d:\gitpybamm\lrh_409\lrh_409\envsolcon\pybamm-gem\pybamm\solvers\casadi_solver.py", line 266, in _integrate
    raise pybamm.SolverError(
pybamm.expression_tree.exceptions.SolverError: Maximum number of decreased steps occurred at t=0.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=7.091632984777611) and/or reducing the size of the time steps or period of the exper

2022-10-06 14:34:23.173 - [WARNING] callbacks.on_experiment_infeasible(233): 

	Experiment is infeasible: 'event: Maximum voltage' was triggered during 'Discharge at 1 C until 2.5 V'. The returned solution only contains up to step 1 of cycle 1. 


Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.5e-11 D_ec= 1.250375e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.5e-11 D_ec= 2.50025e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.5e-11 D_ec= 3.750125e-10
Succeed for:  Xi= -1.0 t_0plus= 0.3 D_cross= 1.5e-11 D_ec= 5e-10


In [ ]:
Model_ddiff.variables.search("potential")

In [28]:
print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

[8.67916594e-08 4.60756039e-01 4.73533886e-01 4.83082544e-01
 4.89548651e-01 4.93048054e-01 4.93670934e-01 4.91486092e-01
 4.86544183e-01 4.78879732e-01 4.68511787e-01 4.55443129e-01
 4.39658034e-01 4.21118612e-01 3.99759666e-01 3.75481755e-01
 3.48141696e-01 3.17538947e-01 2.83394836e-01 2.45318491e-01
 2.23043210e-01 2.20700963e-01 2.18341711e-01 2.15965301e-01
 2.13571577e-01 2.11160381e-01 2.08731547e-01 2.06284910e-01
 2.03820298e-01 2.01337534e-01 1.98836440e-01 1.96316829e-01
 1.93778512e-01 1.91221296e-01 1.88644979e-01 1.86049356e-01
 1.83434217e-01 1.80799344e-01 1.78144515e-01 1.75469499e-01
 1.59917414e-01 1.35649862e-01 1.14942121e-01 9.73908492e-02
 8.26018576e-02 7.02014059e-02 5.98438393e-02 5.12158663e-02
 4.40379640e-02 3.80634899e-02 3.30714740e-02 2.88878424e-02
 2.53899439e-02 2.24857105e-02 2.01048347e-02 1.81932332e-02
 1.67094707e-02 1.56224242e-02 1.49097716e-02 1.45570386e-02]
[1.43882302e-43 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000

In [30]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [35]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=3.254714235538407, step=0.03254714235538407)…

In [32]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    ["EC flux [mol.m-2.s-1]","EC flux by diffusion [mol.m-2.s-1]",
        "EC flux by migration [mol.m-2.s-1]","EC flux by Li+ [mol.m-2.s-1]"],
    ["Li+ flux [mol.m-2.s-1]","Li+ flux by diffusion [mol.m-2.s-1]",
        "Li+ flux by migration [mol.m-2.s-1]","Li+ flux by solvent [mol.m-2.s-1]"],    
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=1,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=3.651721474386636, step=0.03651721474386636)…

In [33]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    "Porosity times concentration",
    "eps_c_e rhs",
    "Minus div Li+ flux",
    "Li+ source term",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=3,
    figsize = (16,12)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=3.651721474386636, step=0.03651721474386636)…

In [ ]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as tanh (0-1).gif")